# Sinergias e Inovações: Explorando as Potencialidades da Intersecção Interdisciplinar entre Ciência de Dados e Geografia

### **Representação Gráfica das Ações das Forças Populares 25 de Abril (FP-25) em Portugal Continental (1980-1987)**

### Disclaimer
Este mapa é apresentado com o objetivo exclusivamente educativo e informativo, visando proporcionar uma visão geral dos incidentes associados às Forças Populares 25 de Abril (FP-25) em Portugal. As informações aqui contidas foram compiladas a partir de fontes consideradas confiáveis e destinam-se a promover a compreensão histórica e a conscientização sobre os eventos relacionados a este grupo durante o período específico da sua atividade.

É importante notar que este mapa não visa promover, glorificar ou desvalorizar quaisquer ações, ideologias ou indivíduos envolvidos, mas apenas uma contextualização de factos históricos descritos nos documentos disponíveis.

Os criadores e colaboradores deste mapa não se responsabilizam pelo uso indevido das informações aqui contidas ou por quaisquer interpretações ou conclusões feitas por terceiros. Encorajamos os usuários a abordarem este material com um espírito crítico e a procurarem fontes adicionais para uma compreensão mais abrangente dos eventos históricos.

- [x] Ao utilizar este mapa, reconhece e concorda com os termos deste disclaimer.

### Introdução
As Forças Populares 25 de Abril (FP-25) emergiram no panorama pós-revolucionário de Portugal, num contexto marcado pela transição democrática após a Revolução dos Cravos de 1974. Esta fase de transformação política e social fomentou a formação de diversos grupos, entre eles as FP-25, constituídas em 1980 pela fusão de organizações de extrema-esquerda com o intuito de promover mudanças sociais segundo princípios socialistas.  O grupo realizou várias ações, incluindo atentados e assaltos, visando influenciar o curso político do país. Estas atividades refletem as tensões da época na luta pela definição da nova ordem democrática. As FP-25 representam uma faceta da complexidade política e social do período pós-revolucionário em Portugal, evidenciando os desafios enfrentados na consolidação da democracia.

### Metodologia
Este projeto parte da curiosidade do alcance das ações das FP-25. Ao descobrir que não havia uma representação gráfica destas ações foi decidido desenvolver uma. Juntando as capacidades e interesses de ambos parceiros, isto é o interesse em História, Geografia e Política, bem como as capacidades em Ciência de Dados e Sistemas de Informação Geográfica; seria um projeto tanto desfrutável, como fazível e dentro dos campos de especialidade de ambos.

Ao analisar a página de Wikipedia dos FP-25 e seguindo para o fim da página encontramos uma lista cronológica dos vários eventos e ações das FP-25, com referências aos artigos e livros em que foram reportas estas ações. 

Utilizando o site Map Marker, foram colocados 123 pontos de interesse, nos locais especificos (sempre que era possível identificá-los). Classificados por cores em 9 categorias: Assalto, Assalto com vitimas mortais, Assalto Falhado, Bombas Detonadas, Bombas Não Detonadas, Assassinatos, Agressão, Fuga da Prisão e Vandalismo. Sendo o título de cada ponto a data da ação em questão (sempre que era possível identificá-la) e a descrição é uma breve descrição da ação ou evento, nos casos de agressão, quem foi agredido e como, nos casos de assalto, quanto dinheiro foi levado (equivalente em $) e nos casos de bombas detonadas, o/s alvo/s e se houve (ou não) vitimas.

Do site foi exportado para um ficheiro .csv os dados georreferenciados das ações que foram importados.

Com a ajuda da biblioteca Pandas do Python, foi feito um pré processamento no DataFrame em causa, ficando com as seguintes colunas:  

	- [x]ID
	- [x]Latitude
	- [x]Longitude
	- [x]Description
	- [x]Color
	- [x]Categoria
	- [x]Data
	- [x]Ano_Mes

Depois disso, com ajuda das bibliotecas **re, datetime, tqdm, plotly.express,** foram feitas as representações gráficas em causa. 

**Todo o código é Open-Source para que possa ser consultado.**


In [28]:
import pandas as pd
import re
from datetime import datetime

import warnings 
warnings.filterwarnings("ignore")

incidents = pd.read_csv('DATA.csv')

Importadas as bibliotecas necessárias e arquivo csv 

In [29]:
incidents['ID'] = range(1, len(incidents) + 1)

incidents.set_index('ID', inplace = True)
incidents.head(1)

,Folder name,Folder color,Latitude,Longitude,Title,Description,Color,Phone number,Timestamp,Pin icon code
ID,,,,,,,,,,
1,NaN,71b300,41.336001,-8.560611,1 de Agosto de 1984,Assalto ao Banco Espírito Santo na Trofa. Foi ...,71b300,NaN,2024-02-08T17:16:00Z,-1


Estado inicial do dataset gerado pelo 	[Map Marker](https://www.mapmarker.app/)

In [30]:
columns_to_drop = ['Folder name', 'Phone number', 'Pin icon code', 'Timestamp', 'Folder color']
incidents.drop(columns=columns_to_drop, inplace = True)

incidents['Color'].replace('ff9800', 'ff5722', inplace=True)

Exclusão de colunas não necessárias e correção de um erro na cor.

In [31]:
incidents['Color'].unique()

array(['71b300', '3f51b5', 'ff5722', 'ffeb3b', 'f44336', '9e9e9e',
       '795548', '9c27b0', '607d8b'], dtype=object)

In [32]:
incident_type = {'71b300' : 'Assalto', 
                '3f51b5': 'Bombas Detonadas',
                'ff5722' : 'Agressao',
                'ffeb3b' : 'Vandalismo',
                'f44336' : 'Assassinato',
                '9e9e9e' : 'Assalto com Vitimas Mortais', 
                '795548' : 'Tentativa de Assalto (falhada)', 
                '9c27b0' : 'Bombas Não Detonadas',
                '607d8b' : 'Fugas da Prisao'
                 }

incidents['Categoria'] = incidents['Color'].map(incident_type)

Criação de um dicionário para criar coluna "categoria"

In [33]:
incidents = incidents.astype({'Color': 'category',  
                          'Categoria': 'category'})

In [34]:
incidents.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
Index: 123 entries, 1 to 123
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype   
---  ------       --------------  -----   
 0   Latitude     123 non-null    float64 
 1   Longitude    123 non-null    float64 
 2   Title        123 non-null    object  
 3   Description  123 non-null    object  
 4   Color        123 non-null    category
 5   Categoria    123 non-null    category
dtypes: category(2), float64(2), object(2)
memory usage: 33.9 KB


Optimização de tipo de dados.

In [35]:
month_mapping = {
    'Janeiro': 1,
    'Fevereiro': 2,
    'Março': 3,
    'Abril': 4,
    'Maio': 5,
    'Junho': 6,
    'Julho': 7,
    'Agosto': 8,
    'Setembro': 9,
    'Outubro': 10,
    'Novembro': 11,
    'Dezembro': 12,
       
    'janeiro': 1,
    'fevereiro': 2,
    'março': 3,
    'abril': 4,
    'maio': 5,
    'junho': 6,
    'julho': 7,
    'agosto': 8,
    'setembro': 9,
    'outubro': 10,
    'novembro': 11,
    'dezembro': 12,
}

def convert_title_to_date(title):

    match = re.search(r'(\d+) de (\w+) de (\d+)', title)
    if match:
        day, month_text, year = match.groups()
        month = month_mapping.get(month_text, 0)
        if month != 0:
            date_str = f"{year}-{month:02d}-{day}"
            return pd.to_datetime(date_str, format="%Y-%m-%d")
    return None


incidents['Data'] = incidents['Title'].apply(convert_title_to_date)
incidents.drop(columns=['Title'], inplace=True)


incidents.head()

,Latitude,Longitude,Description,Color,Categoria,Data
ID,,,,,,
1,41.336001,-8.560611,Assalto ao Banco Espírito Santo na Trofa. Foi ...,71b300,Assalto,1984-08-01
2,38.010202,-7.870127,"Bombas Detonadas, 8 dispositivos no Bairro Ale...",3f51b5,Bombas Detonadas,1985-02-01
3,38.748041,-9.188370,"Agressão, disparos contra administrador de uma...",ff5722,Agressao,1984-06-01
4,39.973942,-8.627772,"Vandalismo, colocação de Pregos na Estrada Nac...",ffeb3b,Vandalismo,1984-05-01
5,38.815583,-9.223445,Assalto à dependência do Crédito Predial Portu...,71b300,Assalto,1983-11-02


Transformação de data de string para date time, formato aceite nos algoritmos depois utilizados.

In [36]:
import plotly.express as px

In [37]:
map_fig = px.scatter_mapbox(incidents,
                            lat='Latitude',
                            lon='Longitude',
                            hover_name='Data',
                            hover_data={'Description': True},
                            color='Categoria',
                            size_max=15,
                            zoom=5.5,
                            height=800,
                            width=1000,
                            center={"lat": 39.5, "lon": -8.0},
                            mapbox_style="open-street-map")

map_fig.update_traces(marker=dict(size=15,)) 

map_fig.show()


Criação do mapa. ( o mapa é interativo )

In [38]:
incidents['Ano_Mes'] = incidents['Data'].dt.to_period('M').astype(str)

incidentes_agrupados = incidents.groupby(['Categoria', 'Ano_Mes']).size().reset_index(name='Contagem')
pivot_data = incidentes_agrupados.pivot(index='Ano_Mes', columns='Categoria', values='Contagem').fillna(0)
pivot_data.reset_index(inplace=True)


melted_data = pivot_data.melt(id_vars=['Ano_Mes'], var_name='Categoria', value_name='Contagem')

fig = px.line(melted_data,
              x='Ano_Mes',
              y='Contagem',
              color='Categoria',
              title='Número de Incidentes por Categoria ao Longo do Tempo',
              labels={'Ano_Mes': 'Data', 'Contagem': 'Número de Incidentes'},
              width=1500,
              height=600,
              template='plotly_white'
              )

fig.show()


In [42]:
import os

# Obtenha o caminho para a pasta de downloads do sistema
downloads_path = os.path.join(os.path.expanduser('~'), 'Downloads')

# Crie o caminho completo para o arquivo CSV na pasta de downloads
csv_file_path = os.path.join(downloads_path, 'fp25.csv')

# Exporte o DataFrame para o arquivo CSV
incidents.to_csv(csv_file_path, index=False)

print(f'O arquivo CSV foi salvo em: {csv_file_path}')


O arquivo CSV foi salvo em: C:\Users\Francisco Batista\Downloads\fp25.csv
